In [1]:
from owlready2 import *
# from emmopy import get_emmo
from neo4j import GraphDatabase, basic_auth

onto_path.append("")
onto_process=get_ontology("C://Abhishek Thesis//ontology//manufacturing.owl").load()

onto_matter=get_ontology("C://Abhishek Thesis//ontology//matter.owl").load()

onto_quantity=get_ontology("C://Abhishek Thesis//ontology//quantities.owl").load()


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# results = onto_quantity.search_one(label="properties.Quantity")
# print(results)

# Creating node-class-EMMO_is_a relations between exisiting classes.

In [3]:

# # To chcek the parent class of a class
# print(onto_quantity['RotationalFrequency'])

In [4]:

# onto_matter.Matter.iri

In [5]:
# print(onto_quantity.Velocity)

## map the 3 ontologies on neo4j

In [6]:



# Define Neo4j connection details
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

# Create Neo4j driver and session
driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

# Define function to create nodes
def create_node(tx, label, name,iri):
    tx.run("MERGE (n:" + label + " {name: $name, uri:$uri})", name=name, uri=iri)
    

# Define function to create EMMO_IS_A relationships
def create_emmo_is_a_rel(tx,class_, subclass, superclass):
    tx.run("MATCH (c1:" + class_ + " {name: $subclass}), (c2:" + class_ + " {name: $superclass}) "
           "MERGE (c1)-[:EMMO_IS_A]->(c2)",class_=class_ ,subclass=subclass, superclass=superclass)

# Create nodes for each ontology
# with session.begin_transaction() as tx:
#     create_node(tx, "EEMOMatter", "Matter")
#     create_node(tx, "EMMOProcess", "Process")
#     create_node(tx, "EMMOQuantity", "Quantity")

# Create nodes for each class in Matter ontology
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        create_node(tx, "EMMOMatter", clazz.name,clazz.iri)

# Create nodes for each class in Process ontology
with session.begin_transaction() as tx:
    for clazz in onto_process.classes():
        create_node(tx, "EMMOProcess", clazz.name,clazz.iri)

# Create nodes for each class in Quantity ontology
with session.begin_transaction() as tx:
    for clazz in onto_quantity.classes():
        create_node(tx, "EMMOQuantity", clazz.name,clazz.iri)

# Create EMMO_IS_A relationships for each class
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOMatter', clazz.name, parent.name)
    for clazz in onto_process.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOProcess', clazz.name, parent.name)
    for clazz in onto_quantity.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOQuantity', clazz.name, parent.name)

# Close Neo4j session and driver
session.close()
driver.close()


# Create separate Node labels and classes

# To create relation between vclasses

In [7]:
# Match(n) optional match(n)-[r]-() delete n,r

# to create relation between vclasses

In [8]:

# query = f"MATCH (m:{node1} {{name: '{domain_cls}'}}), (ma:{node2} {{name: '{range_cls}'}}) CREATE (m)-[:{rel_type}]->(ma)"
#             result = session.run(query)

In [17]:
import pandas as pd
path='C://Abhishek Thesis//OneDrive_1_5-11-2023//test file-organization data//org_data_2.xlsx'
# Read Excel file and specify data types for columns
df = pd.read_excel(path)

data = []
for index, row in df.iterrows():
    attribute = row[0]
    value = row[1]
    data.append((attribute.strip(), str(value).strip()))

# print(data)


# Generate variable names dynamically
var_names = [x[0].lower().replace(' ', '_').replace('-', '_') for x in data]

# Create a string with the assignments
assign_str = '\n'.join([f"{var_names[i]} = '{data[i][1]}'" for i in range(len(data))])

# Execute the string to perform the assignments
exec(assign_str)


Elucidating the Influence of the d-Band Center on the Synthesis of Isobutanol
000000-1
000000-1-3


In [18]:
# def create_or_get_node(tx, label, **properties):
#     """ check for existing nodes, if not there, create new"""
#     query = "MERGE (n:%s {name: $name}) RETURN n" % label
#     result = tx.run(query, name=properties['name'])
#     if not result.peek():
#         create_node(tx, label, **properties)
     
    
    
def create_nodes(tx, label, **properties):
    query = f"CREATE (n:{label}) SET n += $properties RETURN n"
    return tx.run(query, properties=properties)

def create_or_get_node(tx, label, match_property, properties):
    query = f"MERGE (n:{label} {{{match_property}: $value}}) SET n += $properties RETURN n"
    result = tx.run(query, value=properties[match_property], properties=properties)
    if not result.peek():
        create_nodes(tx, label, **properties)


# Function to create the relationship if it doesn't exist
def create_relation(tx, node1, node2, relation, property1, property2):
    query = f"MATCH (n1), (n2) WHERE n1.{property1} = $name1 AND n2.{property2} = $name2 " \
            f"MERGE (n1)-[r:{relation}]->(n2) " \
            f"RETURN count(r)"
    result = tx.run(query, name1=node1, name2=node2)
    count = result.single()[0]
    return count
    
founding_body_list = founding_body.split(",")    
country_list = country.split(",")
institution_list = institution.split(",")
orcid_list = orcid.split(",")
authors_list_list = authors_list.split(",")

# Create nodes for Researcher, Institution and Country
with driver.session() as session:
    with session.begin_transaction() as tx:
    
        for country_ in country_list:
            if country not in ('','-','nan'):
                properties = {'name': country_}
                create_or_get_node(tx, "Country",'name', properties)
        
    
        for institution_ in institution_list:
            if institution not in ('','-','nan'):
                properties = {'name': institution_}
                create_or_get_node(tx, "Institution",'name', properties)

        for authors_list_,orcid_ in zip(authors_list_list,orcid_list):
            if orcid not in ('', '-', 'nan'):
                properties = {'name': authors_list_, 'orcid': orcid_}
                create_or_get_node(tx, "Researcher", 'orcid', properties)

        
#         if orcid not in ('','-','nan'): 
#             create_or_get_node(tx, "Researcher", name=authors_list, orcid=orcid)

                
        
        if subcomponent not in ('','-','nan'):    
            properties = {'name': subcomponent}
            create_or_get_node(tx, "Component",'name', properties)

                
  
        if device not in ('','-','nan'): 
            properties = {'name': device}
            create_or_get_node(tx, "Device",'name', properties)

        

        if material not in ('','-','nan'):  
            properties = {'name': material}
            create_or_get_node(tx, "Material", 'name',properties)

                     

        properties = {'name': measurement_id,'experiment_start':measurement_date,
                    'date_added':upload_date, 'run_title' :experiment_title, 'experiment_id':experiment_id}
        create_or_get_node(tx, "Measurement",'name',properties)

        
        
# config_rel_lst=[{'node1':orcid, 'node2':institution,'relation': 'AFFILIATED_TO', 'property_name1':'orcid','property_name2': 'name'},
#       {'node1':institution, 'node2':country,'relation': 'IN', 'property_name1':'name','property_name2': 'name'},
#       {'node1':orcid, 'node2':country,'relation': 'IN', 'property_name1':'orcid','property_name2': 'name'},
#       {'node1':measurement_id, 'node2':orcid,'relation': 'BY', 'property_name1':'name','property_name2': 'orcid'} 
#      ]
           
# Create relations between the nodes  
    with session.begin_transaction() as tx:  
    # Check and create relationship
        for orcid_,institution_  in zip(orcid_list,institution_list):
            count = create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', 'name')

            if count == 0:
                print("Relationship AFFILIATED_TO doesn't exist, creating it...")
                create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', 'name')
            else:
                pass

        for institution_, country_ in zip(institution_list,country_list):

            count = create_relation(tx, institution_, country_, 'IN', 'name', 'name')

            if count == 0:
                print("Relationship IN doesn't exist, creating it...")
                create_relation(tx, institution_, country_, 'IN', 'name', 'name')
            else:
                pass
        
        
        for orcid_, country_ in zip(orcid_list,country_list):
            count = create_relation(tx, orcid_, country_, 'IN', 'orcid', 'name')

            if count == 0:
                print("Relationship IN_ orcid doesn't exist, creating it...")
                create_relation(tx, orcid_, country_, 'IN', 'orcid', 'name')
            else:
                pass
        
        
        for orcid_ in orcid_list:
            count = create_relation(tx, measurement_id, orcid_, 'BY', 'name', 'orcid')

            if count == 0:
                print("Relationship BY doesn't exist, creating it...")
                create_relation(tx, measurement_id, orcid_, 'BY', 'name', 'orcid')
            else:
                pass
        



67543hb FZJ IEK-13
yuib234 FZJ IEK-14


In [14]:
# print(Measurement_ID,Experiment_ID=Experiment_ID, Experiment_Title=Experiment_Title, Measurement_date=Measurement_date)

In [15]:
# # Define a function to create nodes with properties
# def create_node(tx, label, **properties):
#     query = "CREATE (n:%s) SET n += $properties RETURN n" % label
#     return tx.run(query, properties=properties)

# # Define a function to create a relation between two nodes
# def create_relation(tx, node1, node2, relation):
#     query = "MATCH (n1),(n2) WHERE n1.name = $name1 AND n2.name = $name2 CREATE (n1)-[:%s]->(n2)" % relation
#     return tx.run(query, name1=node1, name2=node2)

# with driver.session() as session:
#     with session.begin_transaction() as tx:
#         # Create nodes for Researcher, Institution, Country, Component, Device and Material
#         if country not in ('', '-', 'nan'):
#             result = tx.run("MERGE (c:Country {name: $country}) RETURN c", country=country)
#             create_node(tx, "Country", name=country)
#         if institution not in ('', '-', 'nan'):
#             result = tx.run("MERGE (c:Institution {name: $institution}) RETURN c", institution=institution)
#             create_node(tx, "Institution", name=institution)
#         if orcid not in ('', '-', 'nan'):
#             result = tx.run("MERGE (c:Researcher {name: $authors_list,orcid:$orcid}) RETURN c", authors_list=authors_list, orcid=orcid)
#             create_node(tx, "Researcher", name=authors_list, orcid=orcid)
#         if subcomponent not in ('', '-', 'nan'):
#             result = tx.run("MERGE (c:Component {name: $subcomponent}) RETURN c", subcomponent=subcomponent)
#             create_node(tx, "Component", name=subcomponent)
#         if device not in ('', '-', 'nan'):
#             result = tx.run("MERGE (c:Device {name: $device}) RETURN c", device=device)
#             create_node(tx, "Device", name=device)
#         if material not in ('', '-', 'nan'):
#             result = tx.run("MERGE (c:Material {name: $material}) RETURN c", material=material)
#             create_node(tx, "Material", name=material)

#         # Create Measurement node with properties and relations to Researcher node
#         create_node(tx, "Measurement", name=measurement_id, experiment_start=measurement_date, date_added=upload_date, run_title=experiment_title, experiment_id=experiment_id)
#         if all(x not in ('', '-', 'nan') for x in [measurement_id, authors_list]):
#             create_relation(tx, measurement_id, authors_list, "BY")

#         # Create relations between nodes
#         if all(x not in ('', '-', 'nan') for x in [authors_list, institution]):
#             create_relation(tx, authors_list, institution, "AFFILIATED_TO")
#         if all(x not in ('', '-', 'nan') for x in [institution, country]):
#             create_relation(tx, institution, country, "IN")
#         if all(x not in ('', '-', 'nan') for x in [authors_list, country]):
#             create_relation(tx, authors_list, country, "IN")
#         if all(x not in ('','-','nan') for x in [measurement_id, authors_list]):    
#             create_relation(tx, measurement_id, authors_list, "BY")

# Create attributes